### DATASET : [kaggle.com/datasets/saurav9786/amazon-product-reviews](https://)

## REV2 IMPLEMENTATION

Se nos entrega un set de datos de reviews de productos de Amazon y buscamos detectar usuarios potencialmente maliciosos y otros que sean honestos. Consideraremos malicioso a un usuario cuya justicia sea menor a 0.2 y tenga al menos 5 reviews. Consideramos extremadamente justos a aquellos usuarios que tengan justicia mayor a 0.9 y con al menos 10 reviews. Para lograr esto utilizaremos el algoritmo REV2.

El set de datos de productos de amazon se compone de cuatro columnas que son las siguientes:

    - ID unico de cada usuario
    
    - ID unico de cada producto
    
    - Rating que el usuario le puso al producto. En el set este valor va de 1 a 5, pero nosotros le aplicaremos la conversión (valor - 3) / 2 para tenerlo entre -1 y 1.
    
    - Timestamp del momento que se hizo el rating.

El algoritmo REV2 recibe como entrada un grafo bipartito con los ratings, donde los nodos son usuarios y productos y las aristas son las reviews con peso entre -1 y 1. 

Cada usuario u tiene una noción de justicia F(u), cada producto p tiene un valor G(p), y cada rating con un usuario u que reseña un producto p tiene una fiabilidad R(u,p). Inicialmente no tenemos ninguno de estos valores así que los seteamos todos en 1. Lo que nosotros queremos obtener son aquellos usuarios u cuyo F(u) sea menor o igual a 0.2 con al menos 5 reviews y aquellos usuarios u con F(u) mayor o igual 0.9 y al menos 10 reviews.

Para calcular los valores pedidos hacemos una clasificación iterativa que se describe de la siguiente forma: Para cada usuario u recorremos todos los productos que rankeo actualizando su F(u) utilizando la sumatoria de la fiabilidad actual de sus ratings. Luego para cada producto p actualizamos su valor G(p) teniendo en cuenta la fiabilidad de sus ratings multiplicado por el score que le puso cada usuario que los rankeo. Por ultimo actualizamos la fiabilidad del ranking que le puso cada usuario a cada producto teniendo en cuenta los parámetros alpha1 y alpha 2, ademas de los que actualizamos en esta iteración: F(u), G(p) y el score de ese usuario a ese producto. 

Repitiendo este procedimiento varias veces podemos lograr un F(u) para cada usuario u que converga al valor correcto y en base a eso decimos si un usuario es malicioso o extremadamente justo. Por supuesto, mientras mas cercano al 0 mas malicioso el usuario y mientras mas cercano al 1, mas justo es, aunque deben haber varios reviews para que esos valores se consideren validos.



In [ ]:
## Code to read csv file into Colaboratory:
#!pip install -U -q PyDrive
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
#from google.colab import auth
#from oauth2client.client import GoogleCredentials
## Authenticate and create the PyDrive client.
#auth.authenticate_user()
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()
#drive = GoogleDrive(gauth)


In [ ]:
#link = 'https://drive.google.com/file/d/1lI2FDIgpSy1jNYsfl7FY9_gBX7g90Big/view?usp=sharing'
#fluff, id = link.split('=')
#print (id) # Verify that you have everything after '='
#id = '1lI2FDIgpSy1jNYsfl7FY9_gBX7g90Big'

In [ ]:
import pandas as pd

In [ ]:
#downloaded = drive.CreateFile({'id':id}) 
#downloaded.GetContentFile('ratings_Electronics.csv')  
df3 = pd.read_csv('ratings_Electronics.csv')
# Dataset is now stored in a Pandas Dataframe


In [ ]:
df3

,AKM1MP6P0OYPR,0132793040,5.0,1365811200
0,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
1,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
2,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
3,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200
4,A1QGNMC6O1VW39,0511189877,5.0,1397433600
...,...,...,...,...
7824476,A2YZI3C9MOHC0L,BT008UKTMW,5.0,1396569600
7824477,A322MDK0M89RHN,BT008UKTMW,5.0,1313366400
7824478,A1MH90R0ADMIK0,BT008UKTMW,4.0,1404172800
7824479,A10M2KEFPEQDHN,BT008UKTMW,4.0,1297555200


In [ ]:
df3.columns = ['userID', 'productID', 'rating', 'timestamp']
df3.columns
df3=df3.append({'userID' : 'AKM1MP6P0OYPR' , 'productID' : '0132793040', 'rating' : '5.0', 'timestamp' : '1365811200'}, ignore_index=True)
df3

C:\Users\jhojman\AppData\Local\Temp\ipykernel_10916\1926420999.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3=df3.append({'userID' : 'AKM1MP6P0OYPR' , 'productID' : '0132793040', 'rating' : '5.0', 'timestamp' : '1365811200'}, ignore_index=True)


,userID,productID,rating,timestamp
0,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
1,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
2,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
3,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200
4,A1QGNMC6O1VW39,0511189877,5.0,1397433600
...,...,...,...,...
7824477,A322MDK0M89RHN,BT008UKTMW,5.0,1313366400
7824478,A1MH90R0ADMIK0,BT008UKTMW,4.0,1404172800
7824479,A10M2KEFPEQDHN,BT008UKTMW,4.0,1297555200
7824480,A2G81TMIOIDEQQ,BT008V9J9U,5.0,1312675200


In [ ]:
df3 = df3.drop(columns=['timestamp'])
df3

,userID,productID,rating
0,A2CX7LUOHB2NDG,0321732944,5.0
1,A2NWSAGRHCP8N5,0439886341,1.0
2,A2WNBOD3WNDNKT,0439886341,3.0
3,A1GI0U4ZRJA8WN,0439886341,1.0
4,A1QGNMC6O1VW39,0511189877,5.0
...,...,...,...
7824477,A322MDK0M89RHN,BT008UKTMW,5.0
7824478,A1MH90R0ADMIK0,BT008UKTMW,4.0
7824479,A10M2KEFPEQDHN,BT008UKTMW,4.0
7824480,A2G81TMIOIDEQQ,BT008V9J9U,5.0


In [ ]:
df3['rating'] = (df3['rating'].astype(float) - 3) / 2
df3

,userID,productID,rating
0,A2CX7LUOHB2NDG,0321732944,1.0
1,A2NWSAGRHCP8N5,0439886341,-1.0
2,A2WNBOD3WNDNKT,0439886341,0.0
3,A1GI0U4ZRJA8WN,0439886341,-1.0
4,A1QGNMC6O1VW39,0511189877,1.0
...,...,...,...
7824477,A322MDK0M89RHN,BT008UKTMW,1.0
7824478,A1MH90R0ADMIK0,BT008UKTMW,0.5
7824479,A10M2KEFPEQDHN,BT008UKTMW,0.5
7824480,A2G81TMIOIDEQQ,BT008V9J9U,1.0


In [ ]:
#df2 = df3.copy()
#df2 = df2.drop(columns=['rating'])
#df2.to_csv('ratings_Electronics_2.csv', index=False)

In [ ]:
import networkx as nx
G = nx.read_edgelist("ratings_Electronics_2.csv", delimiter=",")

In [ ]:
Fu = {}
for userId in df3['userID'].unique():
  Fu[userId] = 1

In [ ]:
Gp = {}
for productId in df3['productID'].unique():
  Gp[productId] = 1

In [ ]:
Rup = {}
for fila in df3.values: 
  Rup[(fila[0], fila[1])] = 1

In [ ]:
rating = {}
for fila in df3.values: 
  rating[(fila[0], fila[1])] = fila[2]

In [ ]:
alpha1 = 0.5
alpha2 = 0.5

In [ ]:
vecinos = nx.to_dict_of_lists(G)

In [ ]:
###### REV2 ALGORITHM ######

for i in range(0, 10):
  print("Iteracion " + str(i))
  for userId in df3['userID'].unique():
    sumRup = 0
    doutu = 0
    for productId in vecinos[userId]:
      sumRup += Rup[(userId, productId)]
      doutu += 1
    Fu[userId] = sumRup / doutu 
  
  for productId in df3['productID'].unique():
    sumRup = 0
    dinp = 0
    for userId in vecinos[productId]:
      sumRup += (Rup[(userId, productId)] * rating[(userId, productId)])
      dinp += 1
    Gp[productId] = sumRup / dinp

  for userId in df3['userID'].unique():
    for productId in vecinos[userId]:
      Rup[(userId, productId)] = ( ( 1/ ( alpha1 + alpha2 ) ) * ( alpha1 * Fu[userId] + alpha2 * ( 1 - ( abs( rating[(userId, productId)] - Gp[productId] ) / 2 ) ) ) )




Iteracion 0
Iteracion 1
Iteracion 2
Iteracion 3
Iteracion 4
Iteracion 5
Iteracion 6
Iteracion 7
Iteracion 8
Iteracion 9


In [ ]:
cant_reviews = df3["userID"].value_counts()
evil_users = {}
extremaly_fair_users = {}
for user in Fu.keys():
  fairness = Fu[user]
  if fairness <= 0.2 and cant_reviews[user] >= 5:
    evil_users[user] = fairness
  if fairness >= 0.9 and cant_reviews[user] >= 10:
    extremaly_fair_users[user] = fairness
print("Los usuarios extremadamente justos son: " + str(extremaly_fair_users))
print("\n\n")
print("Los usuarios maliciosos son: " + str(evil_users))


Los usuarios extremadamente justos son: {'A1LMUHXVNBLTGZ': 0.9171432351546156, 'AVVFMZTW94F8': 0.9065325064102275, 'A3R52J21NWLWW2': 0.9030553568397108, 'AF6XVE84L68BM': 0.9328663275958828, 'AI6CX6EK67FJJ': 0.923457139325409, 'A3DH74L21G9QFJ': 0.9049376622065142, 'A37FULV9NLQYXU': 0.905062503756542, 'A2GBQVP43UVKTA': 0.9066366214486032, 'AWBUA3QUM6DRY': 0.9050337642951767, 'A3EZD11AFUX23K': 0.913672871152447, 'A15BN1AMRT8FP1': 0.9154271877085898, 'A22XSRK69MIRG4': 0.9157944985804288, 'ARFRKSZXZF0PS': 0.9106756797235165, 'A127GZ51PD786F': 0.9008812106460431, 'A2ZNMPZMLRRC8R': 0.9054225910937083, 'A1ISGQPXW3BOHN': 0.9113475121955613, 'AGC75EPFB7CYD': 0.9098135724904981, 'AO49H7W3JR6RL': 0.9022963976719737, 'A2F0XN0L5K7N0T': 0.9293904056929613, 'AY5VJCY3K4DD8': 0.9173370244238613, 'A24KBB6X5URQ63': 0.9008838549371903, 'A1TR1R2QKWRSRA': 0.9110934057372163, 'A3QJX63UDE76OA': 0.9030594729312938, 'AK0PL6SBDEJB9': 0.9052915093930762, 'A29CT3MXKBWVQ6': 0.9269761583097242, 'A150MRVGEWP0UT': 0.90

In [ ]:
print(len(extremaly_fair_users))

416


In [ ]:
###### REV2 ALGORITHM ######

for i in range(0, 50):
  print("Iteracion " + str(i))
  for userId in df3['userID'].unique():
    sumRup = 0
    doutu = 0
    for productId in vecinos[userId]:
      sumRup += Rup[(userId, productId)]
      doutu += 1
    Fu[userId] = sumRup / doutu 
  
  for productId in df3['productID'].unique():
    sumRup = 0
    dinp = 0
    for userId in vecinos[productId]:
      sumRup += (Rup[(userId, productId)] * rating[(userId, productId)])
      dinp += 1
    Gp[productId] = sumRup / dinp

  for userId in df3['userID'].unique():
    for productId in vecinos[userId]:
      Rup[(userId, productId)] = ( ( 1/ ( alpha1 + alpha2 ) ) * ( alpha1 * Fu[userId] + alpha2 * ( 1 - ( abs( rating[(userId, productId)] - Gp[productId] ) / 2 ) ) ) )




Iteracion 0
Iteracion 1
Iteracion 2
Iteracion 3
Iteracion 4
Iteracion 5
Iteracion 6
Iteracion 7
Iteracion 8
Iteracion 9
Iteracion 10
Iteracion 11
Iteracion 12
Iteracion 13
Iteracion 14
Iteracion 15
Iteracion 16
Iteracion 17
Iteracion 18
Iteracion 19
Iteracion 20
Iteracion 21
Iteracion 22
Iteracion 23
Iteracion 24
Iteracion 25
Iteracion 26
Iteracion 27
Iteracion 28
Iteracion 29
Iteracion 30
Iteracion 31
Iteracion 32
Iteracion 33
Iteracion 34
Iteracion 35
Iteracion 36
Iteracion 37
Iteracion 38
Iteracion 39
Iteracion 40
Iteracion 41
Iteracion 42
Iteracion 43
Iteracion 44
Iteracion 45
Iteracion 46
Iteracion 47
Iteracion 48
Iteracion 49


In [ ]:
cant_reviews = df3["userID"].value_counts()
evil_users = {}
extremaly_fair_users = {}
for user in Fu.keys():
  fairness = Fu[user]
  if fairness <= 0.2 and cant_reviews[user] >= 5:
    evil_users[user] = fairness
  if fairness >= 0.9 and cant_reviews[user] >= 10:
    extremaly_fair_users[user] = fairness
print("Los usuarios extremadamente justos son: " + str(extremaly_fair_users))
print("\n\n")
print("Los usuarios maliciosos son: " + str(evil_users))


Los usuarios extremadamente justos son: {'A1LMUHXVNBLTGZ': 0.9159425108410257, 'AVVFMZTW94F8': 0.9051590103452162, 'A3R52J21NWLWW2': 0.9012227174960931, 'AF6XVE84L68BM': 0.9342714573887926, 'AI6CX6EK67FJJ': 0.921170879070631, 'A3DH74L21G9QFJ': 0.9032779950163189, 'A37FULV9NLQYXU': 0.9051377370139855, 'A2GBQVP43UVKTA': 0.9076886077348616, 'AWBUA3QUM6DRY': 0.9066889105201817, 'A3EZD11AFUX23K': 0.9136461203078658, 'A15BN1AMRT8FP1': 0.9153936917399634, 'A22XSRK69MIRG4': 0.915846246677547, 'ARFRKSZXZF0PS': 0.9120542101189845, 'A127GZ51PD786F': 0.901504210377423, 'A2ZNMPZMLRRC8R': 0.9024188415930878, 'A1ISGQPXW3BOHN': 0.9142472575823818, 'AGC75EPFB7CYD': 0.9104858094258028, 'AO49H7W3JR6RL': 0.9030439463597887, 'A2F0XN0L5K7N0T': 0.9295085091622848, 'AY5VJCY3K4DD8': 0.9194160775915865, 'A24KBB6X5URQ63': 0.9017058281232484, 'A1TR1R2QKWRSRA': 0.9117748538007312, 'A3QJX63UDE76OA': 0.9017941132059939, 'AK0PL6SBDEJB9': 0.9043306703670065, 'A29CT3MXKBWVQ6': 0.9282159939860536, 'A150MRVGEWP0UT': 0.90

In [ ]:
print(len(extremaly_fair_users))

407
